In [ ]:
#import sqlalchemy
import sqlalchemy as sqa
from sqlalchemy.orm import declarative_base
from sqlalchemy import func
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy import case
from sqlalchemy import desc

import timeit #for timing
import matplotlib.pyplot as plt
import numpy as np

from sqlalchemy.orm import sessionmaker

In [ ]:
#creates engine to the database file
engine = sqa.create_engine("sqlite:///Data/movie.db")

In [ ]:
#creates declarative base
Base = declarative_base()

In [ ]:
#creates class for communicating with the movie table
class Movies(Base):
    __tablename__ = 'movies'
    
    id_movie = sqa.Column(sqa.Integer, primary_key=True)
    title_movie = sqa.Column(sqa.String)
    year_movie = sqa.Column(sqa.Integer)
    
    def __repr__(self):
        return "<Movie(id_movie='%i',title_movie='%s')>" % (self.id_movie, self.title_movie)

#creates class for communicating with the movie table
class Ratings(Base):
    __tablename__ = 'ratings'
    
    id_rating = sqa.Column(sqa.Integer, primary_key=True)
    id_movie = sqa.Column(sqa.Integer)
    id_user = sqa.Column(sqa.Integer)
    value_rating = sqa.Column(sqa.Float)
    timestamp_rating = sqa.Column(sqa.String)

    
    def __repr__(self):
        return "<Rating(id_rating='%i',id_movie='%i',id_user='%i',value_rating='%i',timestamp_rating='%s')>" % (self.id_rating,self.id_movie,self.id_user,self.value_rating, self.timestamp_rating)

#creates class for communicating with the movie table
class GenreList(Base):
    __tablename__ = 'genre_list'
    
    id_genre_item = sqa.Column(sqa.Integer, primary_key=True)
    id_movie = sqa.Column(sqa.Integer)
    name_genre = sqa.Column(sqa.String)
    
    
    def __repr__(self):
        return "<GenreList(id_genre_item='%i',id_movie='%i',name_genre='%s')>" % (self.id_genre_item,self.id_movie,self.id_user,self.name_genre)

    
#creates class temporary reference of average rating
class ReferenceRatings(Base):
    __tablename__ = 'reference_ratings'
    
    id_reference = sqa.Column(sqa.Integer, primary_key=True)
    average_rating = sqa.Column(sqa.Float)
    timestamp_update = sqa.Column(sqa.String)
    onestar_rating = sqa.Column(sqa.Integer)
    twostar_rating = sqa.Column(sqa.Integer)
    threestar_rating = sqa.Column(sqa.Integer)
    fourstar_rating = sqa.Column(sqa.Integer)
    fivestar_rating = sqa.Column(sqa.Integer)
    count_rating = sqa.Column(sqa.Integer)
    

    
    def __repr__(self):
        return "<Rating(id_movie='%i',average_rating='%i',timestamp_update='%s')>" % (self.id_reference,self.average_rating, self.timestamp_update)

class intersec_movies(Base):
    __tablename__ = 'intersec_movies'
    
    id_intersec = sqa.Column(sqa.Integer, primary_key=True, index=True)
    id_movieA = sqa.Column(sqa.Integer)
    id_movieB = sqa.Column(sqa.Integer)
    count_value = sqa.Column(sqa.Integer)
    
    def __repr__(self):
        return "<intersec_movies(id_moviAe='%i',id_moviAe='%i',count='%i')>" % (self.id_movieA,self.id_movieB, self.count_value)

class intersec_users(Base):
    __tablename__ = 'intersec_users'
    
    id_intersec_user = sqa.Column(sqa.Integer, primary_key=True, index=True)
    id_userA = sqa.Column(sqa.Integer)
    id_userB = sqa.Column(sqa.Integer)
    similarity_user = sqa.Column(sqa.Integer)
    
    def __repr__(self):
        return "<intersec_users(id_intersec_user='%i',id_userA='%i',id_userB='%i', count='%i')>" % (self.id_intersec_user,self.id_userA,self.id_userB, self.similarity_user)

class intersec_movies_2(Base):
    __tablename__ = 'intersec_movies_2'
    
    id_intersec_movie = sqa.Column(sqa.Integer, primary_key=True, index=True)
    id_movieA = sqa.Column(sqa.Integer, index=True)
    id_movieB = sqa.Column(sqa.Integer, index=True)
    similarity_movie = sqa.Column(sqa.Float)
    
    def __repr__(self):
        return "<intersec_movies(id_intersec_movie='%i',id_movieA='%i',id_movieB='%i', similarity='%i')>" % (self.id_intersec_movie,self.id_movieA,self.id_movieB, self.similarity_movie)

    
#relationships

#Ratings.movie = relationship("Movies", back_populates="ratings")
#Movies.ratings = relationship("Ratings", order_by=Ratings.id_rating, back_populates="movie")

In [ ]:
#connects to the database
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
for m in session.query(Movies).order_by(Movies.id_movie)[1:30]:
    print(m.title_movie)

In [ ]:
start = timeit.default_timer()
for m in (session.query(Movies).filter(Movies.title_movie.like("%Jumanji%Welcome%"))[0:30]):
    print(m.title_movie)

stop = timeit.default_timer()

print('Time: ', stop - start) 

In [ ]:
query = session.query(ReferenceRatings.id_reference, ReferenceRatings.average_rating, ReferenceRatings.count_rating)

In [ ]:
query_arr = np.asarray(query[:])

In [ ]:
query_arr[1]

In [ ]:
sample_average = query_arr[:,1]
sample_counts = query_arr[:,2]

x = np.argsort(sample_average)
sample_average = np.sort(sample_average)
sample_counts = sample_counts[x]

In [ ]:
sample_counts[5000]

In [ ]:
sample[2]

In [ ]:
sample = np.sort(sample)

In [ ]:
start = timeit.default_timer() #start timer

max_movie = session.query(func.max(Movies.id_movie)).all()[0][0] + 1 #number of movies

aavg = np.zeros(max_movie)
acount = np.zeros(max_movie)
aone = np.zeros(max_movie)
atwo = np.zeros(max_movie)
athree = np.zeros(max_movie)
afour = np.zeros(max_movie)
afive = np.zeros(max_movie)

#queries for temporary information
one_star = session.query(Ratings.id_movie, func.count(Ratings.value_rating)).filter(Ratings.value_rating <= 1).group_by(Ratings.id_movie).order_by(Ratings.id_movie)
two_star = session.query(Ratings.id_movie, func.count(Ratings.value_rating)).filter(Ratings.value_rating <= 2, Ratings.value_rating > 1).group_by(Ratings.id_movie).order_by(Ratings.id_movie) 
three_star = session.query(Ratings.id_movie, func.count(Ratings.value_rating)).filter(Ratings.value_rating <= 3, Ratings.value_rating > 2).group_by(Ratings.id_movie).order_by(Ratings.id_movie)
four_star = session.query(Ratings.id_movie, func.count(Ratings.value_rating)).filter(Ratings.value_rating <= 4, Ratings.value_rating > 3).group_by(Ratings.id_movie).order_by(Ratings.id_movie)
five_star = session.query(Ratings.id_movie, func.count(Ratings.value_rating)).filter(Ratings.value_rating <= 5, Ratings.value_rating > 4).group_by(Ratings.id_movie).order_by(Ratings.id_movie)
average = session.query(Ratings.id_movie, func.avg(Ratings.value_rating), func.count(Ratings.value_rating)).group_by(Ratings.id_movie).order_by(Ratings.id_movie)


list_values = zip(average,one_star,two_star,three_star,four_star,five_star)

for item in list_values:
    aavg[item[0][0]] = item[0][1]
    acount[item[0][0]] = item[0][2]
    aone[item[1][0]] = item[1][1]
    atwo[item[2][0]] = item[2][1]
    athree[item[3][0]] = item[3][1]
    afour[item[4][0]] = item[4][1]
    afive[item[5][0]] = item[5][1]
    

stop = timeit.default_timer() #end timer

print('Time: ', stop - start) 

In [ ]:
#create list for number of ratings and average

aavg = np.zeros

start = timeit.default_timer() #start timer

list_values = zip(average,one_star,two_star,three_star,four_star,five_star)

for item in list_values:
    #print("MovieID:" + str(item[0][0]) + " AVG:" + str(item[0][1]) + " COUNT:" + str(item[0][2]))
    id_1 = item[0][0]
    id_2 = item[1][0]
    id_3 = item[2][0]
    id_4 = item[3][0]
    id_5 = item[4][0]
    id_6 = item[5][0]

stop = timeit.default_timer() #end timer

print('Time: ', stop - start) 

In [ ]:
start = timeit.default_timer() #start timer

query = session.query(Movies.id_movie, ReferenceRatings.average_rating,).filter(Movies.id_movie == ReferenceRatings.id_reference).order_by(Movies.id_movie)[0:30]
print(query[:10])

stop = timeit.default_timer() #end timer

print('Time: ', stop - start) 

In [ ]:
start = timeit.default_timer() #start timer
for m in session.query(Movies.title_movie, Movies.year_movie,ReferenceRatings.average_rating).filter(Movies.id_movie == ReferenceRatings.id_reference).order_by(Movies.id_movie)[0:30]:
    print(m)
stop = timeit.default_timer() #end timer

print('Time: ', stop - start) 

In [ ]:
start = timeit.default_timer()
for m in (session.query(GenreList.name_genre.distinct())):
    print(m[0].strip('\n'))

stop = timeit.default_timer()

print('Time: ', stop - start) 

In [ ]:
start = timeit.default_timer()
genre_query = session.query(Movies.title_movie, Movies.year_movie,ReferenceRatings.average_rating, ReferenceRatings.count_rating, GenreList.id_movie, GenreList.name_genre).filter(Movies.id_movie == ReferenceRatings.id_reference, Movies.id_movie == GenreList.id_movie).order_by(ReferenceRatings.count_rating.desc())
genre_query = genre_query.filter(GenreList.name_genre == "Animation")
genre_query[0:10]
stop = timeit.default_timer()
print('Time: ', stop - start) 

In [ ]:
genre_query[0:10]

In [ ]:
#get the recommendation for a certain movie

movieid = 2

list_movies = []
list_count = []

intersecA = session.query(intersec_movies).filter(intersec_movies.id_movieA == movieid)
for row in intersecA:
    list_movies.append(row.id_movieB)
    list_count.append(row.count_value)

intersecB = session.query(intersec_movies).filter(intersec_movies.id_movieB == movieid)
for row in intersecB:
    list_movies.append(row.id_movieA)
    list_count.append(row.count_value)
    
list_movies = np.asarray(list_movies)
list_count = np.asarray(list_count)

list_movies = np.flip(list_movies[np.argsort(list_count)])
list_count = np.flip(np.sort(list_count))


In [ ]:
for i in range(0,10):
    print("ID:" + str(list_movies[i]))
    print("COUNT:" + str(list_count[i]))

In [ ]:
session.query(intersec_movies).filter(intersec_movies.id_movieA == movieid).count()

In [ ]:
query = session.query(intersec_movies).filter(intersec_movies.id_movieA == 1)

In [ ]:
query.count()

In [ ]:
query.delete()

In [ ]:
session.commit()

In [ ]:
query = session.query(intersec_users.id_userA, intersec_users.id_userB, intersec_users.similarity_user).order_by(intersec_users.similarity_user.desc())

In [ ]:
query[100:110]

In [ ]:
query.count()

In [ ]:
#get the recommendation for a certain user
#100260
userid = 100260

list_users = []
list_sim = []

intersecA = session.query(intersec_users).filter(intersec_users.id_userA == userid)
for row in intersecA:
    list_users.append(row.id_userB)
    list_sim.append(row.similarity_user)

intersecB = session.query(intersec_users).filter(intersec_users.id_userB == userid)
for row in intersecB:
    list_users.append(row.id_userA)
    list_sim.append(row.similarity_user)
    
list_users = np.asarray(list_users)
list_count = np.asarray(list_sim)

list_users = np.flip(list_users[np.argsort(list_sim)])
list_sim = np.flip(np.sort(list_sim))

In [ ]:
for i in range(0,10):
    print("ID:" + str(list_users[i]))
    print("COUNT:" + str(list_sim[i]))

In [ ]:
list_users[0]

In [ ]:
recommend_1 = session.query(Ratings.id_movie).filter(Ratings.id_user.in_((int(list_users[0]),int(list_users[1]),int(list_users[2]))), Ratings.value_rating >= 3.5).all()
#recommend_2 = session.query(Ratings.id_movie).filter(Ratings.id_user == int(list_users[1]), Ratings.value_rating >= 3.5).all()
#recommend_3 = session.query(Ratings.id_movie).filter(Ratings.id_user == int(list_users[2]), Ratings.value_rating >= 3.5).all()
userrated = session.query(Ratings.id_movie).filter(Ratings.id_user == 100260).all()

In [ ]:
rlist_1 = []
#rlist_2 = []
#rlist_3 = []
ulist = []

for r in recommend_1:
    rlist_1.append(r[0])

#for r in recommend_2:
#    rlist_2.append(r[0])

#for r in recommend_3:
#    rlist_3.append(r[0])

for l in userrated:
    ulist.append(l[0])

In [ ]:
recommended_set = set(rlist_1).union(set(rlist_2)).union(set(rlist_3)) - set(ulist)

In [ ]:
len(recommended_set)

In [ ]:
fullrlist = []
fullrlist_names = []
relevanceindex = []

for movie in recommended_set:
    query = session.query(ReferenceRatings.average_rating).filter(ReferenceRatings.id_reference == movie)
    query2 = session.query(Movies.title_movie).filter(Movies.id_movie == movie)
    fullrlist.append(movie)
    fullrlist_names.append(query2[0][0])
    relevanceindex.append(query[0][0])

fullrlist = np.asarray(fullrlist)
fullrlist_names = np.asarray(fullrlist_names)
relevanceindex = np.asarray(relevanceindex)

In [ ]:
fullrlist = np.flip(fullrlist[np.argsort(relevanceindex)])
fullrlist_names = np.flip(fullrlist_names[np.argsort(relevanceindex)])
relevanceindex = np.flip(np.sort(relevanceindex))

In [ ]:
for i in range(0,100):
    print("ID:" + str(fullrlist[i]))
    print("TITLE:" + str(fullrlist_names[i]))
    print("AVG:" + str(relevanceindex[i]))